# Treemap
Z pomocą biblioteki **pyplot** wykonano wykres, na którym widać proporcjonalnie przychody największych franczyz na świecie (stan na dzień 15.12.2025)

In [32]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.io as pio

import textwrap
import base64

Korzystamy z pliku franchise_worth.xlsx, w którym znajdują się dane z https://en.wikipedia.org/wiki/List_of_highest-grossing_media_franchises dla 19 największych franczyz (stan na dzień 15.12.2025)

In [33]:
df=pd.read_excel('franchise_worth.xlsx')
df

,id,franchise,year of inception,total revenue (est. US$),original medium,creator,owner
0,1,Pokémon,1996,115000000000,Video game,Satoshi Tajiri,The Pokémon Company; Nintendo
1,2,Mickey Mouse & Friends,1928,61200000000,Animated cartoon,Walt Disney; Ub Iwerks,The Walt Disney Company
2,3,Winnie the Pooh,1924,50200000000,Book,A. A. Milne; E. H. Shepard,The Walt Disney Company
3,4,Star Wars,1977,46700000000,Film,George Lucas,Lucasfilm (The Walt Disney Company)
4,5,Disney Princess,2000,45400000000,Animated films,Andy Mooney,The Walt Disney Company
5,6,Anpanman,1973,38300000000,Picture book,Takashi Yanase,Froebel-kan (Toppan Printing Co.)
6,7,Barbie,1987,36300000000,Television special,Ruth Handler,Mattel
7,8,Marvel Cinematic Universe,2008,35200000000,Film,Marvel Studios; Stan Lee; Jack Kirby,"Walt Disney Studios (The Walt Disney Company),..."
8,9,Wizarding World (Harry Potter),1997,34700000000,Novel,J. K. Rowling,"J. K. Rowling (books), Warner Bros. (Warner Br..."
9,10,Hello Kitty,1974,33500000000,Cartoon character,Yuko Shimizu; Shintaro Tsuji,Sanrio


In [34]:
df['clean revenue'] = df['total revenue (est. US$)']/1e9
df_plot = df.sort_values('total revenue (est. US$)', ascending=False).head(15)
df_plot['wrapped label'] = df_plot['franchise'].apply(lambda x: '<br>'.join(textwrap.wrap(x, width=15)))
df_plot['revenue text'] = df_plot['clean revenue'].apply(lambda x: f'{x:.1f} B')
df_plot['log revenue'] = np.log10(df_plot['clean revenue'])
df_plot


,id,franchise,year of inception,total revenue (est. US$),original medium,creator,owner,clean revenue,wrapped label,revenue text,log revenue
0,1,Pokémon,1996,115000000000,Video game,Satoshi Tajiri,The Pokémon Company; Nintendo,115.0,Pokémon,115.0 B,2.060698
1,2,Mickey Mouse & Friends,1928,61200000000,Animated cartoon,Walt Disney; Ub Iwerks,The Walt Disney Company,61.2,Mickey Mouse &<br>Friends,61.2 B,1.786751
2,3,Winnie the Pooh,1924,50200000000,Book,A. A. Milne; E. H. Shepard,The Walt Disney Company,50.2,Winnie the Pooh,50.2 B,1.700704
3,4,Star Wars,1977,46700000000,Film,George Lucas,Lucasfilm (The Walt Disney Company),46.7,Star Wars,46.7 B,1.669317
4,5,Disney Princess,2000,45400000000,Animated films,Andy Mooney,The Walt Disney Company,45.4,Disney Princess,45.4 B,1.657056
5,6,Anpanman,1973,38300000000,Picture book,Takashi Yanase,Froebel-kan (Toppan Printing Co.),38.3,Anpanman,38.3 B,1.583199
12,13,Spider-Man,1962,36800000000,Comic book,Stan Lee; Steve Ditko,Marvel Entertainment (The Walt Disney Company)...,36.8,Spider-Man,36.8 B,1.565848
6,7,Barbie,1987,36300000000,Television special,Ruth Handler,Mattel,36.3,Barbie,36.3 B,1.559907
7,8,Marvel Cinematic Universe,2008,35200000000,Film,Marvel Studios; Stan Lee; Jack Kirby,"Walt Disney Studios (The Walt Disney Company),...",35.2,Marvel<br>Cinematic<br>Universe,35.2 B,1.546543
8,9,Wizarding World (Harry Potter),1997,34700000000,Novel,J. K. Rowling,"J. K. Rowling (books), Warner Bros. (Warner Br...",34.7,Wizarding World<br>(Harry Potter),34.7 B,1.540329


Z uwagi na problemy z zapisywaniem plików przy pomocy Kaleido na Macbook M1 Pro, należało zmienić ścieżkę obrazków na format Base64. Mamy w kodzie dwa obrazki - dlatego stworzyłyśmy funkcję, która zwraca ścieżkę w poprawnynm formacie.

In [35]:
def get_image_source(path):
    #..plik czytamy binarnie - obrazki to ciąg zer i jedynek, bez "rb" python próbowałby odczytać dane jako litery
    with open(path, "rb") as image_file: 
        #..base64 zmienia dane binarne na kod ASCII, decode() zmienia bajty na string
        encoded_string = base64.b64encode(image_file.read()).decode()
    #..data URI scheme
    return "data:image/png;base64," + encoded_string

## Tworzenie wykresu w plotly
W plotly z reguły tworzy się interaktywne wykresy z Plotly.js. Tutaj jednak będziemy chcieli zapisać plik jako png. 
Użyjemy skali logarytmicznej do pokolorowania całej mapy, aby było widoczne większe rozróżnienie między kolorami

In [48]:
fig = px.treemap(
    df_plot, 
    px.Constant(" "),
    path=[px.Constant(" "), 'franchise'], 
    values='clean revenue',
    custom_data=['revenue text', 'wrapped label'],
    color='log revenue', 
    color_continuous_scale='Blues',
)

fig.add_layout_image(
    dict(
        source=get_image_source('images/images/143.png'),
        xref="paper", yref="paper", # Pozycjonowanie względem całego płótna
        x=1.01, y=0.11,  
        sizex=0.12, sizey=0.12, # Rozmiar obrazka (współczynnik 0-1)
        xanchor="left", yanchor="top", # Punkt zakotwiczenia obrazka
        layer="above" # Obrazek ma być na wierzchu
    )
)

fig.add_layout_image(
    dict(
        source=get_image_source('franchise_images/1.png'),
        xref="paper", yref="paper", # Pozycjonowanie względem całego płótna
        x=0.15, y=0.917,  
        sizex=0.2, sizey=0.2, # Rozmiar obrazka (współczynnik 0-1)
        xanchor="left", yanchor="top", # Punkt zakotwiczenia obrazka
        layer="above" # Obrazek ma być na wierzchu
    )
)

fig.update_traces(
    # Używamy owiniętej nazwy zamiast oryginalnego label
    texttemplate="<b>%{customdata[1]}</b><br>%{customdata[0]}",
    textposition="middle center",
    textfont_size=15,
    tiling=dict(packing='squarify'),
    marker=dict(
        pad=dict(t=5, l=5, r=5, b=5),  # Zerujemy odstęp wewnętrzny
        line=dict(width=2, color='white') # Ustawiamy białą ramkę między kafelkami
    )
)

fig.update_layout(
    title_text='Top 15 Highest-Grossing Franchises (US$)',
    title_x=0.5,
    margin=dict(t=50, l=10, r=10, b=10),
    width=1600,  
    height=900, 
    uniformtext=dict(minsize=12, mode='show'),
    coloraxis_showscale=False
)

#pio.write_image(fig, "franchise_worth.png", scale=3) #trzeba mieć zainstalowane kaleido i google chrome
fig.show()

In [50]:
fig = px.treemap(
    df_plot, 
    px.Constant(" "),
    path=[px.Constant(" "), 'franchise'], 
    values='clean revenue',
    custom_data=['revenue text', 'wrapped label'],
    color='clean revenue', 
    color_continuous_scale='Blues',
)

fig.add_layout_image(
    dict(
        source=get_image_source('images/images/143.png'),
        xref="paper", yref="paper", # Pozycjonowanie względem całego płótna
        x=1.01, y=0.11,  
        sizex=0.12, sizey=0.12, # Rozmiar obrazka (współczynnik 0-1)
        xanchor="left", yanchor="top", # Punkt zakotwiczenia obrazka
        layer="above" # Obrazek ma być na wierzchu
    )
)

fig.add_layout_image(
    dict(
        source=get_image_source('franchise_images/1.png'),
        xref="paper", yref="paper", # Pozycjonowanie względem całego płótna
        x=0.15, y=0.917,  
        sizex=0.2, sizey=0.2, # Rozmiar obrazka (współczynnik 0-1)
        xanchor="left", yanchor="top", # Punkt zakotwiczenia obrazka
        layer="above" # Obrazek ma być na wierzchu
    )
)

fig.update_traces(
    # Używamy owiniętej nazwy zamiast oryginalnego label
    texttemplate="<b>%{customdata[1]}</b><br>%{customdata[0]}",
    textposition="middle center",
    textfont_size=15,
    tiling=dict(packing='squarify'),
    marker=dict(
        pad=dict(t=5, l=5, r=5, b=5),  # Zerujemy odstęp wewnętrzny
        line=dict(width=2, color='white') # Ustawiamy białą ramkę między kafelkami
    )
)

fig.update_layout(
    title_text='Top 15 Highest-Grossing Franchises (US$)',
    title_x=0.5,
    margin=dict(t=50, l=10, r=10, b=10),
    width=800,  
    height=800, 
    uniformtext=dict(minsize=12, mode='show'),

    coloraxis=dict(
        colorbar_title_text='Revenue<br>($B)', # <br> łamie linię
        colorbar_title_side='top', 
        colorbar_len=0.9,
        colorbar_yanchor="top", colorbar_y=1,
    )   
)

#pio.write_image(fig, "franchise_worth.png", scale=3) #trzeba mieć zainstalowane kaleido i google chrome
fig.show()

In [37]:

fig = px.treemap(
    df_plot, 
    px.Constant(" "),
    path=[px.Constant(" "), 'franchise'], 
    values='clean revenue',
    custom_data=['revenue text', 'wrapped label'],
    color='clean revenue', 
    color_continuous_scale='Blues',
)

fig.add_layout_image(
    dict(
        source=get_image_source('images/images/143.png'),
        xref="paper", yref="paper", # Pozycjonowanie względem całego płótna
        x=1.01, y=0.11,  
        sizex=0.12, sizey=0.12, # Rozmiar obrazka (współczynnik 0-1)
        xanchor="left", yanchor="top", # Punkt zakotwiczenia obrazka
        layer="above" # Obrazek ma być na wierzchu
    )
)

fig.add_layout_image(
    dict(
        source=get_image_source('franchise_images/1.png'),
        xref="paper", yref="paper", # Pozycjonowanie względem całego płótna
        x=0.034, y=0.817,  
        sizex=0.2, sizey=0.2, # Rozmiar obrazka (współczynnik 0-1)
        xanchor="left", yanchor="top", # Punkt zakotwiczenia obrazka
        layer="above" # Obrazek ma być na wierzchu
    )
)

fig.update_layout(
    title_text='Franchise Worth (Top 15, US$)',
    title_x=0.5,
    margin=dict(t=50, l=10, r=10, b=10),
    width=800,  
    height=500, 
    uniformtext=dict(minsize=8, mode='show'),

    coloraxis=dict(
        colorbar_title_text='Revenue<br>(B$)', # <br> łamie linię
        colorbar_title_side='top', 
        colorbar_len=0.9,
        colorbar_yanchor="top", colorbar_y=1,
    )   
)

fig.update_traces(
    # Używamy owiniętej nazwy zamiast oryginalnego label
    texttemplate="<b>%{customdata[1]}</b><br>%{customdata[0]}",
    textposition="middle center",
    textfont_size=15,
    tiling=dict(packing='squarify'),
    marker=dict(
        pad=dict(t=5, l=5, r=5, b=5),  # Zerujemy odstęp wewnętrzny
        line=dict(width=2, color='white') # Ustawiamy białą ramkę między kafelkami
    )
)


#pio.write_image(fig, "franchise_worth2.png", scale=3) #trzeba mieć zainstalowane kaleido i google chrome
fig.show()